In [47]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [48]:
df = pd.read_csv("Dataset/ESSAY_SURVEY.csv");
df.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,Unnamed: 7,rater1_domain2,rater2_domain2,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,Answer Activities,NaN,NaN,NaN,3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"i open it up, play genshin impact, after that,...",NaN,NaN,NaN,5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,Utilizing my computer productively as a studen...,NaN,NaN,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,I spend a significant amount of time coding we...,NaN,NaN,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,Probably the productive things I do with my co...,NaN,NaN,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 29 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   essay_id        18 non-null     int64  
 1   essay_set       18 non-null     int64  
 2   essay           18 non-null     object 
 3   rater1_domain1  0 non-null      float64
 4   rater2_domain1  0 non-null      float64
 5   rater3_domain1  0 non-null      float64
 6   domain1_score   18 non-null     int64  
 7   Unnamed: 7      0 non-null      float64
 8   rater1_domain2  0 non-null      float64
 9   rater2_domain2  0 non-null      float64
 10  domain2_score   0 non-null      float64
 11  rater1_trait1   0 non-null      float64
 12  rater1_trait2   0 non-null      float64
 13  rater1_trait3   0 non-null      float64
 14  rater1_trait4   0 non-null      float64
 15  rater1_trait5   0 non-null      float64
 16  rater1_trait6   0 non-null      float64
 17  rater2_trait1   0 non-null      float

In [50]:
df.shape

(18, 29)

In [51]:
df.isnull().sum()

essay_id           0
essay_set          0
essay              0
rater1_domain1    18
rater2_domain1    18
rater3_domain1    18
domain1_score      0
Unnamed: 7        18
rater1_domain2    18
rater2_domain2    18
domain2_score     18
rater1_trait1     18
rater1_trait2     18
rater1_trait3     18
rater1_trait4     18
rater1_trait5     18
rater1_trait6     18
rater2_trait1     18
rater2_trait2     18
rater2_trait3     18
rater2_trait4     18
rater2_trait5     18
rater2_trait6     18
rater3_trait1     18
rater3_trait2     18
rater3_trait3     18
rater3_trait4     18
rater3_trait5     18
rater3_trait6     18
dtype: int64

In [52]:
df[df['essay_set']==7]

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,Unnamed: 7,rater1_domain2,rater2_domain2,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6


In [53]:
df.dropna(axis=1,inplace=True)
df.shape

(18, 4)

In [54]:
df.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,Answer Activities,3
1,2,1,"i open it up, play genshin impact, after that,...",5
2,3,1,Utilizing my computer productively as a studen...,8
3,4,1,I spend a significant amount of time coding we...,10
4,5,1,Probably the productive things I do with my co...,7


In [55]:
df['essay_set'].value_counts()

essay_set
1    6
2    6
3    6
Name: count, dtype: int64

In [57]:
# df.drop(columns=['rater1_domain1','rater2_domain1'],inplace=True,axis=1)
df.dropna(subset=['essay', 'domain1_score'], inplace=True)

# Exclude the 'essay' column
numeric_df = df.select_dtypes(include='number')

In [58]:
numeric_df.shape

(18, 3)

In [59]:
numeric_df.head()

,essay_id,essay_set,domain1_score
0,1,1,3
1,2,1,5
2,3,1,8
3,4,1,10
4,5,1,7


In [60]:
min(numeric_df['domain1_score'])

2

In [61]:
max(numeric_df['domain1_score'])

10

In [62]:
numeric_df.skew()

essay_id         0.000000
essay_set        0.000000
domain1_score   -0.229716
dtype: float64

In [63]:
numeric_df.kurtosis()

essay_id        -1.20000
essay_set       -1.59375
domain1_score   -1.38369
dtype: float64

In [64]:
numeric_df[numeric_df['essay_set']==2]['domain1_score']

6     2
7     3
8     9
9     7
10    8
11    4
Name: domain1_score, dtype: int64

In [65]:
min_range = [2,1,0,0,0,0,0,0]
max_range = [12,6,3,3,4,4,30,60]

def normalize(x,mi,ma):
    #print("Before Normalization: "+str(x))
    x = (x-mi)/(ma-mi)
    #print("After Normalization : "+str(x))
    return round(x*10)

numeric_df['final_score']=numeric_df.apply(lambda x:normalize(x['domain1_score'],min_range[x['essay_set']-1],max_range[x['essay_set']-1]),axis=1)

In [66]:
numeric_df.drop('domain1_score',axis=1,inplace=True)

In [67]:
# Bring back the 'essay' column
df = pd.concat([df.drop(columns=numeric_df.columns, errors='ignore'), numeric_df], axis=1)
df.head()

,essay,domain1_score,essay_id,essay_set,final_score
0,Answer Activities,3,1,1,1
1,"i open it up, play genshin impact, after that,...",5,2,1,3
2,Utilizing my computer productively as a studen...,8,3,1,6
3,I spend a significant amount of time coding we...,10,4,1,8
4,Probably the productive things I do with my co...,7,5,1,5


In [68]:
def clean_essay(essay):
    x=[]
    for i in essay.split():
        if i.startswith("@"):
            continue
        else:
            x.append(i)
    return ' '.join(x)

df['essay'] = df['essay'].apply(lambda x:clean_essay(x))

In [69]:
stop_words = set(stopwords.words('english')) 
def remove_stop_words(essay):
    word_tokens = word_tokenize(essay) 
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    return ' '.join(filtered_sentence)

df['clean_essay'] = df['essay'].apply(lambda x:remove_stop_words(x))

In [70]:
def remove_puncs(essay):
    essay = re.sub("[^A-Za-z ]","",essay)
    return essay

df['clean_essay'] = df['clean_essay'].apply(lambda x:remove_puncs(x))

In [71]:
df.head()

,essay,domain1_score,essay_id,essay_set,final_score,clean_essay
0,Answer Activities,3,1,1,1,Answer Activities
1,"i open it up, play genshin impact, after that,...",5,2,1,3,open play genshin impact open honkai star r...
2,Utilizing my computer productively as a studen...,8,3,1,6,Utilizing computer productively student involv...
3,I spend a significant amount of time coding we...,10,4,1,8,I spend significant amount time coding website...
4,Probably the productive things I do with my co...,7,5,1,5,Probably productive things I computer I spent ...


In [72]:
def sent2word(x):
    x=re.sub("[^A-Za-z0-9]"," ",x)
    words=nltk.word_tokenize(x)
    return words

def essay2word(essay):
    essay = essay.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw = tokenizer.tokenize(essay)
    final_words=[]
    for i in raw:
        if(len(i)>0):
            final_words.append(sent2word(i))
    return final_words
        

def noOfWords(essay):
    count=0
    for i in essay2word(essay):
        count=count+len(i)
    return count

def noOfChar(essay):
    count=0
    for i in essay2word(essay):
        for j in i:
            count=count+len(j)
    return count

def avg_word_len(essay):
    return noOfChar(essay)/noOfWords(essay)

def noOfSent(essay):
    return len(essay2word(essay))

def count_pos(essay):
    sentences = essay2word(essay)
    noun_count=0
    adj_count=0
    verb_count=0
    adverb_count=0
    for i in sentences:
        pos_sentence = nltk.pos_tag(i)
        for j in pos_sentence:
            pos_tag = j[1]
            if(pos_tag[0]=='N'):
                noun_count+=1
            elif(pos_tag[0]=='V'):
                verb_count+=1
            elif(pos_tag[0]=='J'):
                adj_count+=1
            elif(pos_tag[0]=='R'):
                adverb_count+=1
    return noun_count,verb_count,adj_count,adverb_count

data = open('big.txt').read()
words = re.findall('[a-z]+', data.lower())

def check_spell_error(essay):
    essay=essay.lower()
    new_essay = re.sub("[^A-Za-z0-9]"," ",essay)
    new_essay = re.sub("[0-9]","",new_essay)
    count=0
    all_words = new_essay.split()
    for i in all_words:
        if i not in words:
            count+=1
    return count
    
    

In [73]:
pro_data = df.copy()
pro_data['char_count'] = pro_data['essay'].apply(noOfChar)
pro_data['word_count'] = pro_data['essay'].apply(noOfWords)
pro_data['sent_count'] = pro_data['essay'].apply(noOfSent)
pro_data['avg_word_len'] = pro_data['essay'].apply(avg_word_len)
pro_data['spell_err_count'] = pro_data['essay'].apply(check_spell_error)
pro_data['noun_count'], pro_data['adj_count'], pro_data['verb_count'], pro_data['adv_count'] = zip(*pro_data['essay'].map(count_pos))
pro_data.to_csv("Processed_data.csv")

In [74]:
prep_df = pd.read_csv("Processed_data.csv")
prep_df.drop('Unnamed: 0',inplace=True,axis=1)
prep_df.head()

,essay,domain1_score,essay_id,essay_set,final_score,clean_essay,char_count,word_count,sent_count,avg_word_len,spell_err_count,noun_count,adj_count,verb_count,adv_count
0,Answer Activities,3,1,1,1,Answer Activities,16,2,1,8.000000,0,2,0,0,0
1,"i open it up, play genshin impact, after that,...",5,2,1,3,open play genshin impact open honkai star r...,241,60,2,4.016667,7,16,15,6,2
2,Utilizing my computer productively as a studen...,8,3,1,6,Utilizing computer productively student involv...,410,68,4,6.029412,12,22,13,6,3
3,I spend a significant amount of time coding we...,10,4,1,8,I spend significant amount time coding website...,287,51,3,5.627451,9,15,12,6,2
4,Probably the productive things I do with my co...,7,5,1,5,Probably productive things I computer I spent ...,269,52,1,5.173077,8,16,13,5,1


In [75]:
from sklearn.preprocessing import LabelEncoder

In [82]:
# vectorizer = CountVectorizer(max_features = 10000, ngram_range=(1, 3), stop_words='english')
# count_vectors = vectorizer.fit_transform(df['clean_essay'])
# feature_names = vectorizer.get_feature_names_out()
# data = df[['essay_set','clean_essay','final_score']].copy()
# X = count_vectors.toarray()
# y = data['final_score'].values
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)



# Vectorize the text data
vectorizer = CountVectorizer(max_features=10000, ngram_range=(1, 3), stop_words='english')
count_vectors = vectorizer.fit_transform(prep_df['clean_essay'])
X_text = count_vectors.toarray()

# Get additional features (ensure all are numeric)
additional_features = prep_df.iloc[:, 5:]
# Check if there are any non-numeric columns and encode them
for col in additional_features.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    additional_features[col] = le.fit_transform(additional_features[col])

X_additional = additional_features.values

# Combine text features with additional features
X_full = np.concatenate((X_additional, X_text), axis=1)

# Get the target variable
y_full = prep_df['final_score'].values

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.3, random_state=42)



SVR

In [83]:
#Save Trained Model
clf = SVR(C=1.0, epsilon=0.2)
clf.fit(X_train, y_train)
pickle.dump(clf,open("Saved_Models/SVR_with_pp",'wb'))

#Use Saved Model
# clf = pickle.load(open('Saved_Models/SVR_with_pp', 'rb'))
# y_pred=clf.predict(X_test)
# print("Mean squared error:%.2f"%mean_squared_error(y_test,y_pred))

In [84]:
y_pred=clf.predict(X_test)
print("Mean squared error:%.2f"%mean_squared_error(y_test,y_pred))

Mean squared error:51.52


In [85]:
df.head()

,essay,domain1_score,essay_id,essay_set,final_score,clean_essay
0,Answer Activities,3,1,1,1,Answer Activities
1,"i open it up, play genshin impact, after that,...",5,2,1,3,open play genshin impact open honkai star r...
2,Utilizing my computer productively as a studen...,8,3,1,6,Utilizing computer productively student involv...
3,I spend a significant amount of time coding we...,10,4,1,8,I spend significant amount time coding website...
4,Probably the productive things I do with my co...,7,5,1,5,Probably productive things I computer I spent ...


In [93]:
df = pd.read_csv("Dataset/dummy_data.csv");
df.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
df.shape

(1, 28)

In [95]:
pro_data = df.copy()
pro_data['char_count'] = pro_data['essay'].apply(noOfChar)
pro_data['word_count'] = pro_data['essay'].apply(noOfWords)
pro_data['sent_count'] = pro_data['essay'].apply(noOfSent)
pro_data['avg_word_len'] = pro_data['essay'].apply(avg_word_len)
pro_data['spell_err_count'] = pro_data['essay'].apply(check_spell_error)
pro_data['noun_count'], pro_data['adj_count'], pro_data['verb_count'], pro_data['adv_count'] = zip(*pro_data['essay'].map(count_pos))
pro_data.to_csv("Processed_data.csv")

prep_df = pd.read_csv("Processed_data.csv")
prep_df.drop('Unnamed: 0',inplace=True,axis=1)
prep_df.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater3_trait6,char_count,word_count,sent_count,avg_word_len,spell_err_count,noun_count,adj_count,verb_count,adv_count
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,1483,350,16,4.237143,11,83,74,18,24


In [96]:
from sklearn.preprocessing import LabelEncoder
# Vectorize the text data
vectorizer = CountVectorizer(max_features=10000, ngram_range=(1, 3), stop_words='english')
count_vectors = vectorizer.fit_transform(prep_df['clean_essay'])
X_text = count_vectors.toarray()


KeyError: 'clean_essay'

In [92]:
clf = pickle.load(open('Saved_Models/SVR_with_pp', 'rb'))
clf.predict(count_vectors)

d:\GitHub_repositories\pythonProject\LSTM\Automatic-Essay-Scoring\.venv\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: "Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenager is always on the phone with friends! Do you ever time to chat with your friends or buisness partner about things. Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: @ORGANIZATION1, @ORGANIZATION2, @CAPS1, facebook, myspace ect. Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. How did you learn about other countrys/states outside of yours? Well I have by computer/internet, it's a new way to learn about what going on in our time! You might think your child spends a lot of time on the comp